In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import numpy as np
import imp
import os

In [3]:
def get_best_error(infile):
    
    index = []
    best_fit = []
    bound = []
    
    f = open(infile, 'r')
    lines = f.readlines()
    
    for line in lines:
        if len(line.split())>4:
            if line.split()[-2] == '+/-':
                index.append(line.split()[0])
                best_fit.append(line.split()[-3])
    best = dict(zip(index,best_fit)) 
    
    index = []
    
    for line in lines:
        if len(line.split())>3:
            if line.split()[-1][-1]==")":
                if len(line.split()[-1].split(','))>1.1:
                    index.append(line.split()[0])
                    bound.append([line.split()[1], line.split()[2]])

    bounds = dict(zip(index,bound))
    
    
    f.close()
    
    return best, bounds

# infile is for the best-fit table and error result both, 
# just copy the result of 'show free','show fit' and 'error' in xspec. 

In [4]:
def get_par_best_error(infile):
    
    index = []
    parameter = []
    best_fit = []
    bound = []
    
    f = open(infile, 'r')
    lines = f.readlines()
    
    for line in lines:
        if len(line.split())>4: 
            if line.split()[-2] == '+/-':
                if (line.split()[3] == 'norm' or line.split()[3] == 'logxi'):
                    index.append(line.split()[0])
                    parameter.append(line.split()[3] + "_" + line.split()[2])
                else:
                    index.append(line.split()[0])
                    parameter.append(line.split()[3])
    parameters = dict(zip(index,parameter))
    
    
    for line in lines:
        if len(line.split())>4: 
            if line.split()[-2] == '+/-':
                index.append(line.split()[0])
                best_fit.append(line.split()[-3])
    best = dict(zip(index,best_fit)) 
    
    
    for line in lines:
        if len(line.split())>3:
            if line.split()[-1][-1]==")":
                if len(line.split()[-1].split(','))>1.1:
                    index.append(line.split()[0])
                    bound.append([line.split()[1], line.split()[2]])
                    
    bounds = dict(zip(index,bound))
    
    
    f.close()
    
    return parameters, best, bounds

In [5]:
def get_chi_sq_single_file(infile):
    
    f = open(infile, 'r')
    lines = f.readlines()
    
    index = ['chisq','dof','reduced_chisq']
    value = [None]*3
    
    for line in lines:
        if len(line.split())>4:
            if line.split()[-1] == 'd.o.f.':
                value[0] = float(line.split()[3])
                value[1] = int(line.split()[-2])
                value[2] = float("{:.6f}".format(value[0] / value[1]))
    
    result = dict(zip(index,value))
    
    return result

In [6]:
def get_latex_elements_single_file(infile):
    import imp
    import error2latex as el
    imp.reload(el)
    
    parameter, best_fit, bounds = get_par_best_error(infile)
    
    #index = []
    par = []
    latex_c = []
    
    if len(best_fit)!=len(bounds):
        print("Error: best-fit parameters do not match the number of uncertainties")
        print(f"Number of free parameters: {len(best_fit)}, number of boundaries: {len(bounds)}")
    
    for key in best_fit.keys():
        name = parameter[key]
        best = best_fit[key]
        lower = bounds[key][0]
        upper = bounds[key][1]
        string, value_base, err_low_base, err_high_base = el.get_xspec_error(best, lower, upper) 
        
        #index.append(key)
        par.append(name)
        latex_c.append(string)    
        
       # print(string)
    #result = dict(zip(index,latex_c))
    result = dict(zip(par,latex_c))
    
    return result

In [7]:
def get_latex_elements_multiple_file(infiles):
    number = len(infiles)
    eles = [None]*number
    for i in range(number):
        eles[i] = get_latex_elements_single_file(infiles[i])
    pars = []
    for i in range(number):
        pars = pars + list(eles[i].keys())
    keys_origin = list(set(pars))
    keys = sort_parameters(keys_origin)
    
    return eles, keys

In [8]:
def sort_parameters(key_origin):
    parameter_lib = ['nH',
                     'norm_nthComp',
                     'Gamma','kT_e',
                     'Index1','Index2','Index3','Rbr1','Rbr2','a','Incl','logxi_relxillCp','Afe','norm_relxillCp',
                     'logxi_xillverCp','norm_xillverCp',
                     'factor'] #在此函数中设定表格中参数的顺序，根据需求修改
    a = range(len(parameter_lib))
    dict1 = dict(zip(a,parameter_lib))
    dict2 = dict(zip(parameter_lib,a))
    key_temp_1 = []
    keys = []
    
    for i in key_origin:
        key_temp_1.append(dict2.get(i))
        
    key_temp_2 = sorted(key_temp_1)  
    
    for j in key_temp_2:
        keys.append(dict1.get(j))
        
    return keys

In [9]:
def print_chi_sq_multiple_files(infiles):
    
    number = len(infiles)
    chisq, dof, reduced_chisq = [None]*number, [None]*number,[None]*number
    for i in range(number):
        chisq[i] = get_chi_sq_single_file(infiles[i]).get('chisq')
        dof[i] = get_chi_sq_single_file(infiles[i]).get('dof')
        reduced_chisq[i] = get_chi_sq_single_file(infiles[i]).get('reduced_chisq')

    string = '$\chi^2$ /dof'
    
    for n in range(number):
        string = string + ' & ' +'\\tabincell{c}{' + str(chisq[n]) + '/' + str(dof [n]) + '=\\\\' + str(reduced_chisq[n]) + '}'
        
    print('\\hline' + '\n\n' + string + '\\\\' + '\n\n' + '\\hline' + '\n')

In [10]:
#创建一个字典，键是序列keys里的元素，值是对应的latex字符串
#需根据实际情况进行扩充或修改。
def get_latex_of_parameters(keys):
    number = len(keys)
    keys_latex = [None]*number
    for i in range(number):
        if keys[i] == 'nH':
            keys_latex[i] = '$N_{\\rm H}$ [\\(10^{22}\\) cm\\(^{-2}\\)]'
        if keys[i] == 'norm_nthComp':
            keys_latex[i] = 'Norm({\\tt nthComp})'
        if keys[i] == 'Gamma':
            keys_latex[i] = '$\\Gamma$'
        if keys[i] == 'kT_e':
            keys_latex[i] = '$kT_{\\rm e}$ [keV]'
        if keys[i] == 'Index1':
            keys_latex[i] = '$q_{\\rm in}$'
        if keys[i] == 'Index2':
            keys_latex[i] = '$q_{\\rm out}$ '
        if keys[i] == 'Index3':
            keys_latex[i] = 'Index3'
        if keys[i] == 'Rbr1':
            keys_latex[i] = '$R_{\\rm br1}$ [$R_{\\rm g}$]'
        if keys[i] == 'Rbr2':
            keys_latex[i] = '$R_{\\rm br2}$ [$R_{\\rm g}$]'
        if keys[i] == 'a':
            keys_latex[i] = '$a_{*}$'
        if keys[i] == 'Incl':
            keys_latex[i] = '$i$ [deg]'
        if keys[i] == 'logxi_relxillCp':
            keys_latex[i] = 'log${\\xi}$ [erg~cm~s$^{-1}$]'
        if keys[i] == 'logxi_xillverCp':
            keys_latex[i] = 'log${\\xi}$ [erg~cm~s$^{-1}$]'   
        if keys[i] == 'Afe':
            keys_latex[i] = 'A$_{\\rm Fe}$'
        if keys[i] == 'norm_relxillCp':
            keys_latex[i] = 'Norm({\\tt relxillCp})'
        if keys[i] == 'norm_xillverCp':
            keys_latex[i] = 'Norm({\\tt xillverCp})'
        if keys[i] == 'factor':
            keys_latex[i] = '$C_{\\rm HXD}$ '
    par_latex = dict(zip(keys,keys_latex))
    return par_latex

In [11]:
def updateFile_NK(file):
    # 替换文件中的字符串,把rexill_nk的后缀nk去掉。这个函数对需要把relxill和relxill_nk放在同一行比较时才有用。
    file_data = ""
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            if 'relxillCp_nk' in line:
                line = line.replace('relxillCp_nk','relxillCp ')
            file_data += line
    with open(file,"w",encoding="utf-8") as f:
        f.write(file_data)

In [12]:
def updateFile_Rbr(file):
    # 替换文件中的字符串,把Rbr替换为Rbr_1。这个函数对需要把relxill和relxill_nk放在同一行比较时才有用。
    file_data = ""
    with open(file, "r", encoding="utf-8") as f:
        for line in f:
            if ('Rbr' in line) and ('Rbr1' not in line) and ('Rbr2' not in line):
                line = line.replace('Rbr','Rbr1')
            file_data += line
    with open(file,"w",encoding="utf-8") as f:
        f.write(file_data)

In [13]:
def updateFiles_NK(infiles):
    for i in range(len(infiles)):
        updateFile_NK(infiles[i]) 

In [14]:
def updateFiles_Rbr(infiles):
    for i in range(len(infiles)):
        updateFile_Rbr(infiles[i]) 

In [23]:
def print_final_latex_code(infiles):
    
    updateFiles_NK(infiles) #文件预处理，有时候并不需要，请注释掉。
    updateFiles_Rbr(infiles) #文件预处理，有时候并不需要，请注释掉。
    
    eles, keys = get_latex_elements_multiple_file(infiles)
    
    par_latex = get_latex_of_parameters(keys)
    
    print('\\newcommand{\\tabincell}[2]{\\begin{tabular}{@{}#1@{}}#2\\end{tabular}}' + '\n'
          + '%please copy this line to the front of your .tex document in order to use \\tabincell'+ '\n\n' )
    
    print('\\begin{table*}' + '\n' + '\\centering' + '\n' + '\\renewcommand\\arraystretch{1.5}' + '\n'
          + '\\caption{}' + '\n' + '\\label{bestfit}' + '\n' + '\\resizebox{\\textwidth}{70mm}{' )

    chart = '' 
    for i in range(len(infiles)):
        chart += 'c'

    print('\\begin{tabular}{l' + chart + '}'+ '\n')
    
    print('\\hline' + '\n')
    
    model = 'Model'
    for i in range(len(infiles)):
        model += (' & ' + 'Model ' + str(i+1))
    
    print(model + '\\\\' + '\n')
    
    print('\\hline' + '\n')
    
    for par in keys:
        string = par_latex.get(par)
        for n in range(len(infiles)):
            if par in set(list(eles[n].keys())):
                string = string + ' & '+ eles[n][par]
            else:
                string = string + ' & ' + ' - '
        print(string + '\\\\' + '\n')

    print_chi_sq_multiple_files(infiles)
    
    print('\\end{tabular}' + '\n' + '}' + '\n' + '\\end{table*}')

In [24]:

infiles = ['fit1.txt',
           'fit2.txt',
           'fit3.txt',
           'fit4.txt',
           'fit5.txt',
          'fit6.txt',
          'fit7.txt',
          'fit8.txt']

#在执行前，请根据实际情况修改函数：sort_parameters()，get_latex_of_parameters()，print_final_latex_code()
print_final_latex_code(infiles)

\newcommand{\tabincell}[2]{\begin{tabular}{@{}#1@{}}#2\end{tabular}}
%please copy this line to the front of your .tex document in order to use \tabincell


\begin{table*}
\centering
\renewcommand\arraystretch{1.5}
\caption{}
\label{bestfit}
\resizebox{\textwidth}{70mm}{
\begin{tabular}{lcccccccc}

\hline

Model & Model 1 & Model 2 & Model 3 & Model 4 & Model 5 & Model 6 & Model 7 & Model 8\\

\hline

$N_{\rm H}$ [\(10^{22}\) cm\(^{-2}\)] & $9.08_{-0.07}^{+0.05}$ & $8.98_{-0.08}^{+0.04}$ & $9.04_{-0.05}^{+0.04}$ & $7.34_{-0.15}^{+0.1}$ & $8.62_{-0.05}^{+0.07}$ & $7.3_{-0.1}^{+0.1}$ & $8.878_{-0.05}^{+0.029}$ & $7.30_{-0.08}^{+0.07}$\\

Norm({\tt nthComp}) & $4.76_{-0.22}^{+0.12}$ & $4.72_{-0.4}^{+0.11}$ & $4.71_{-0.19}^{+0.14}$ & $0.43_{-0.12}^{+0.09}$ & $4.53_{-0.3}^{+0.22}$ & $0.68_{-0.3}^{+0.28}$ & $4.68_{-0.1}^{+0.04}$ & $0.43_{-0.1}^{+0.08}$\\

$\Gamma$ & $2.442_{-0.011}^{+0.004}$ & $2.419_{-0.01}^{+0.008}$ & $2.424_{-0.009}^{+0.009}$ & $1.951_{-0.022}^{+0.02}$ & $2.400_{-0.004}^{+